Aufgabe 1: Dimensionen und Fakten

Facts (Numerical measures that can be aggregated):

    Air_Time (minutes)
    Elapsed_Time (minutes)
    Scheduled_Time (minutes)
    Taxi_Out (minutes)
    Taxi_In (minutes)

    Distance (miles)

    Departure_Delay (minutes)
    Arrival_Delay (minutes)
    Air_System_Delay (minutes)
    Security_Delay (minutes)
    Airline_Delay (minutes)
    Late_Aircraft_Delay (minutes)
    Weather_Delay (minutes)

Dimensions

Time Dimension (Temporal Hierarchy):

    Year
    Month
    Day
    Day_of_Week
    Scheduled_Departure 
    Scheduled_Arrival
    Wheels_On
    Wheels_Off
    Arrival_Time

Airline Dimension:

    IATA_CODE (Identifier)
    Airline (Name)

Flight Dimension:

    Flight_ID
    Flight_Number
    Tail_Number

Flight Status Dimension:
    
    Diverted
    Cancelled


Airport Dimension:

    IATA_CODE (Location Identifier)
    Airport (Name)
    Latitude
    Longitude
    City
    State
    Country




Dimension Schema:
![Demensions Schema](./dimensionsSchema.png)

Aufgabe 2: Konzeptionelle M/ER Modell

Conceptual Model:
![Conceptual Model](./conceptModel.png)

Aufgabe 3: Logical Model

Logical Model:
![Star Schema](./logicalModel.png)

Aufgabe 4: RelationaleTabellen (DDL Statements)

```sql
-- Date dimension
CREATE DIMENSION time_dim
    LEVEL day IS (
        dim_date.day_id,
        dim_date.day
    )
    LEVEL month IS (
        dim_date.month_id,
        dim_date.month
    )
    LEVEL quarter IS (
        TRUNC((dim_date.month_id-1)/3) + 1 AS quarter_id,
        'Q' || TO_CHAR(TRUNC((dim_date.month_id-1)/3) + 1) AS quarter
    )
    LEVEL year IS (
        dim_date.year_id,
        dim_date.year
    )
    HIERARCHY time_rollup (
        day    CHILD OF
        month  CHILD OF
        quarter CHILD OF
        year
    );

-- Airport dimension
CREATE DIMENSION location_dim
    LEVEL city IS (
        dim_airport.city_id,
        dim_airport.city
    )
    LEVEL state IS (
        dim_airport.state_id,
        dim_airport.state
    )
    LEVEL country IS (
        dim_airport.country_id,
        dim_airport.country
    )
    HIERARCHY geo_rollup (
        city   CHILD OF
        state  CHILD OF
        country
    );

-- date dimension tables
CREATE TABLE dim_date (
    date_id INTEGER PRIMARY KEY,
    day_id SMALLINT,
    day VARCHAR(10),
    month_id SMALLINT,
    month VARCHAR(10),
    year_id SMALLINT,
    year VARCHAR(10),
    day_of_week SMALLINT
);

-- fact table
CREATE TABLE fact_flights (
    flight_id BIGINT PRIMARY KEY,
    date_id INTEGER REFERENCES dim_date(date_id),
    airline_id INTEGER REFERENCES dim_airline(airline_id),
    status_id SMALLINT REFERENCES dim_status(status_id),
    original_airport_id INTEGER REFERENCES dim_airport(airport_id),
    destination_airport_id INTEGER REFERENCES dim_airport(airport_id),
    flight_number VARCHAR(10),
    tail_number VARCHAR(10),
    scheduled_time TIMESTEP,
    scheduled_departure TIMESTEP,
    scheduled_arrival TIMESTEP,
    distance INTEGER,
    departure_time TIMESTEP,
    wheels_off TIMESTEP,
    wheels_on TIMESTEP,
    taxi_out INTERVAL,
    taxi_in INTERVAL,
    air_time INTERVAL,
    elapsed_time INTERVAL,
    arrival_delay SMALLINT,
    departure_delay SMALLINT,
    air_system_delay SMALLINT,
    airline_delay SMALLINT,
    security_delay SMALLINT,
    weather_delay SMALLINT,
    late_aircraft_delay SMALLINT,
    DIMENSION time_dim REFERENCES date_id,
    DIMENSION location_dim REFERENCES original_airport_id
);

--declare the dimension tables
ALTER DIMENSION time_dim
    DETERMINE day BY (day_id, day)
    DETERMINE month BY (month_id, month)
    DETERMINE quarter BY (quarter_id, quarter)
    DETERMINE year BY (year_id, year);

ALTER DIMENSION location_dim
    DETERMINE city BY (city_id, city)
    DETERMINE state BY (state_id, state)
    DETERMINE country BY (country_id, country);
```
